In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from keras.preprocessing import image
from PIL import Image

### Conversion function from PIL to OpenCV format
def pil2opencv(in_image):
    out_image = np.array(in_image, dtype=np.uint8)
    return out_image

### Conversion function from OpenCV to PIL format
def opencv2pil(in_image):
    new_image = in_image.copy()   # Create a copy of the image
    new_image = Image.fromarray(new_image)
    return new_image

### Function to ensure image is square
def square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        result = pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), (background_color))
        result.paste(pil_img, (0, (width - height) // 2))
    else:
        result = Image.new(pil_img.mode, (height, height), (background_color))
        result.paste(pil_img, ((height - width) // 2, 0))
    return result

### Function to apply Hough Circle Transform and create masks
def hough(img):
    # Apply median filter to the image
    img_median = cv2.medianBlur(img, 15)

    # Apply Hough Circle Transform to detect circles
    circles = cv2.HoughCircles(img_median, cv2.HOUGH_GRADIENT, 1.5, 100,
                                param1=150, param2=100, minRadius=50, maxRadius=300)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        img_edges_hough = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
        # Draw all detected circles
        for i in circles[0,:]:
            cv2.circle(img_edges_hough, (i[0], i[1]), i[2], (0, 255, 0), 2)  # Outer circle
            cv2.circle(img_edges_hough, (i[0], i[1]), 2, (0, 0, 255), 3)  # Center of circle
        
        # Extract the largest circle
        HARD_LIMIT_DIM = 288
        biggestCircle = [0, 0, 0]
        for i in circles[0,:]:
            z = i[2]
            if (z > biggestCircle[2]) and (z < HARD_LIMIT_DIM):
                biggestCircle = i
        
        # Draw the largest circle
        cv2.circle(img_edges_hough, (biggestCircle[0], biggestCircle[1]), biggestCircle[2], (255, 0, 0), 10)
        cv2.circle(img_edges_hough, (biggestCircle[0], biggestCircle[1]), 2, (255, 0, 0), 10)
        
        # Create mask for the largest circle
        outerRad = int(biggestCircle[2])
        shape = img_edges_hough.shape[1::-1]
        outerCircle = np.zeros((shape[1], shape[0]), dtype=np.uint8)
        cv2.circle(outerCircle, center=(biggestCircle[0], biggestCircle[1]), radius=outerRad, color=255, thickness=-1)
        ori[outerCircle == 0] = [255]
        
    return ori

DEBUG = False
HARD_LIMIT_DIM = 500

folder = ['Image']  # Directory for images
Class = ['Retry', 'Pass']  # Classification classes
image_size_x = 288  # Input image width
image_size_y = 216  # Input image height

#path = r'.\Image\2*.jpg'
#filenames = glob.glob(path)
#print(filenames)

for index, name in enumerate(folder):
    dir = name
    files = glob.glob(dir + "/*.jpg")  # List of image files in the directory
    print(files)

labels = ['all_mean','i_mean','m_mean','o_mean','all_std','i_std','m_std','o_std','all_edge_mean','i_edge_mean','m_edge_mean','o_edge_mean','all_edge_std','i_edge_std','m_edge_std','o_edge_std']

df  = pd.DataFrame(columns=labels)

#for filename in filenames:
for filename in files:
    # Read image
    img = cv2.imread(filename, 0)
    print(filename)

    # Resize image
    img_resized = cv2.resize(img, dsize=None, fx=0.25, fy=0.25)
    cv2.imwrite('img.jpg', img_resized, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

    print('size:{}'.format(img_resized.size))
    print('shape:{}'.format(img_resized.shape))

    # Apply median filter
    img_median = cv2.medianBlur(img_resized, 15)

    # Apply Hough Circle Transform
    circles = cv2.HoughCircles(img_median, cv2.HOUGH_GRADIENT, 1.5, 50,
                               param1=150, param2=100, minRadius=200, maxRadius=300)

    if circles is not None:
        # Draw all detected circles
        circles = np.uint16(np.around(circles))
        img_edges_hough = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2BGR)
        for i in circles[0,:]:
            cv2.circle(img_edges_hough, (i[0], i[1]), i[2], (0, 255, 0), 2)  # Outer circle
            cv2.circle(img_edges_hough, (i[0], i[1]), 2, (0, 0, 255), 3)  # Center of circle
        cv2.imwrite('img_edges_hough.png', img_edges_hough)

        # Extract the largest circle
        biggestCircle = [0, 0, 0]
        for i in circles[0,:]:
            z = i[2]
            if (z > biggestCircle[2]) and (z < HARD_LIMIT_DIM):
                biggestCircle = i
        
        print('biggest circle')
        print(biggestCircle)
        
        # Draw the largest circle
        cv2.circle(img_edges_hough, (biggestCircle[0], biggestCircle[1]), biggestCircle[2], (0, 0, 255), 10)
        cv2.circle(img_edges_hough, (biggestCircle[0], biggestCircle[1]), 2, (0, 0, 255), 10)

        # Create masks for inner, middle, and outer regions
        shape = img_edges_hough.shape[1::-1]
        interRad = int(0.57350362 * biggestCircle[2])
        midRad = int(0.816496821 * biggestCircle[2])
        outerRad = int(biggestCircle[2])
        print('radius')
        print(interRad, midRad, outerRad)

        interMask = np.zeros((shape[1], shape[0]), dtype=np.uint8)
        cv2.circle(interMask, center=(biggestCircle[0], biggestCircle[1]), radius=interRad, color=255, thickness=-1)

        midCircle = np.zeros((shape[1], shape[0]), dtype=np.uint8)
        cv2.circle(midCircle, center=(biggestCircle[0], biggestCircle[1]), radius=midRad, color=255, thickness=-1)

        outerCircle = np.zeros((shape[1], shape[0]), dtype=np.uint8)
        cv2.circle(outerCircle, center=(biggestCircle[0], biggestCircle[1]), radius=outerRad, color=255, thickness=-1)

        midMask = ~interMask & midCircle
        outerMask = ~midCircle & outerCircle

        norm = cv2.normalize(img_resized, None, alpha=5, beta=250, norm_type=cv2.NORM_MINMAX, mask=outerCircle)
        outer = norm.copy()
        inner = norm.copy()
        middle = norm.copy()
        outer = norm.copy()
        inner[interMask == 0] = [255]
        middle[midMask == 0] = [255]
        outer[outerMask == 0] = [255]

        plt.axis('off')
        plt.imshow(cv2.cvtColor(inner, cv2.COLOR_GRAY2BGR))
        plt.savefig("inner_pub.tif", format="tif", dpi=300)
        plt.axis('off')
        plt.imshow(cv2.cvtColor(middle, cv2.COLOR_GRAY2BGR))
        plt.savefig("middle_pub.tif", format="tif", dpi=300)
        plt.axis('off')
        plt.imshow(cv2.cvtColor(outer, cv2.COLOR_GRAY2BGR))
        plt.savefig("outer_pub.tif", format="tif", dpi=300)

    # Create an image data generator
    params = {'rotation_range': 180, 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'vertical_flip': True, 'horizontal_flip': True, 'shear_range': 2}
    datagen = image.ImageDataGenerator(**params)
    ori = Image.open(filename).convert("L")  # Read the image
    ori = ori.resize((image_size_x, image_size_y))  # Resize the image
    ori = square(ori, 255)
    plt.imshow(ori)
    plt.axis('off')
    plt.gray()
    plt.tight_layout()
    plt.savefig("img_crop_pub.tif", format="tif", dpi=300)
    ori = pil2opencv(ori)
    hough(ori)
    ori = opencv2pil(ori)

    data = np.asarray(ori)  # Convert image data to array
    x = data[np.newaxis]  # (Height, Width) -> (1, Height, Width) 
    x = x[:, :, :, np.newaxis]  # (1, Height, Width) -> (1, Height, Width, 1) 
    gen = datagen.flow(x, batch_size=1)  # Generate mini-batch with size 1
    
    # Generate and display 9 images using the Python generator
    for n in range(0, 9):  # Specify the number of images
        batches = next(gen)  # Return (NumBatches, Height, Width, Channels)
        aug = np.squeeze(batches)

        if DEBUG:
            gen_img = batches[0].astype(np.uint8)
            plt.imshow(gen_img)
            plt.axis('off')
            plt.gray()
            plt.tight_layout()
            plt.savefig(f"img_dataaug_pub{n:04d}.tif", format="tif", dpi=300)

    plt.show()

    cv2.imwrite('image_resized.png', img_resized)
    cv2.imwrite('img_edges_hough_biggest.png', img_edges_hough)
    cv2.imwrite('mask.png', norm)

    if DEBUG:
        plt.axis('off')
        plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_GRAY2BGR))
        plt.savefig("img_resized_pub.tif", format="tif", dpi=300)
        plt.axis('off')
        plt.imshow(cv2.cvtColor(img_edges_hough, cv2.COLOR_BGR2RGB))
        plt.savefig("img_circle_pub.tif", format="tif", dpi=300)
        plt.imshow(cv2.cvtColor(img_masked, cv2.COLOR_GRAY2BGR))
        plt.savefig("img_masked_pub.tif", format="tif", dpi=300)

        plt.tight_layout()
        plt.show()

# Violin plot
plt.style.use('default')
sns.set()
sns.set_style('white')
sns.set_palette('bright')

df = pd.read_csv('violinplot.csv', index_col=0)
df = df.drop(columns=['raw(s)', 'inner(s)', 'middle(s)', 'outer(s)', 'result'])
df = df.rename(columns={'raw(m)': 'raw', 'inner(m)': 'inner', 'middle(m)': 'middle', 'outer(m)': 'outer'})

fig = plt.figure(figsize=[11, 8.5])
ax = fig.add_subplot(1, 1, 1)
sns.violinplot(data=df, ax=ax, height=1, aspect=1.6)
fontsize = 30
ax.set_ylabel('Mean value (-)', fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.xticks(fontsize=fontsize)
ax.set_ylim(0, 255)
sns.set(font_scale=1, style='ticks')
plt.savefig("img_violin_pub.tif", format="tif", dpi=300)
plt.show()

# Load data for training
data = pd.read_csv("svm.csv", sep=",")

# Independent variables
columns = ['raw(m)', 'inner(m)', 'middle(m)', 'outer(m)', 'raw(s)', 'inner(s)', 'middle(s)', 'outer(s)']
data_x = data.loc[:, columns].values

# Dependent variable
data_y = data['result'].values

pp = data.dropna(subset=['result'])
pp = pp.replace({'result': {0: 'NG', 1: 'OK'}})
pp = pp.rename(columns={'result': 'Decision'})

pp1 = sns.pairplot(pp, hue='Decision', markers=['X', 'o'], vars=columns, height=1.0, aspect=1)
pp1._legend.remove()
pp1.savefig('pairplot.tif', format="tif", dpi=300)
plt.show()



[]


FileNotFoundError: [Errno 2] No such file or directory: 'violinplot.csv'